In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!ls

drive  sample_data


In [4]:
import os

In [5]:
os.chdir("drive/MyDrive/cse 257 Search & Optimization/GraphQSat_modified/")

In [ ]:
!pip3 install --upgrade pip

In [6]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch_scatter    -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch_sparse      -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch_cluster     -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch_spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch_geometric 

Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 2.6MB 32.6MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 1.5MB 29.1MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 1.0MB 31.1MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
     |████████████████████████████████| 389kB 386kB/s 
     |████████████████████████████████| 215kB 33.1MB/s 
     |████████████████████████████████| 235kB 41.7MB/s 
     |████████████████████████████████| 2.2MB 32.6MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.7.0-cp37-none-any.whl size=365386 sha256=edb92436ce12f1b7d6ba1fccb62022a7dfa83c6cefdd556d8e67b568ef5e6141
  Stored in directory: /root/.cache/pip/wheels/ce/fb/df/37cd43d1433e65b2d3417a7143840

In [ ]:
!chmod +x train.sh

In [7]:
!pip3 install tensorboardX

     |████████████████████████████████| 122kB 31.4MB/s 


In [ ]:
!pip install easydict

In [8]:
from easydict import EasyDict as edict

#### Paramters

In [ ]:
!ls

add_metadata.py  docker_run.sh	gqsat			     minisat
data		 dqn.py		graph-env		     README.md
docker_build.sh  evaluate.py	GraphQSAT_driver_code.ipynb  runs
Dockerfile	 evaluate.sh	LICENSE			     train.sh


In [ ]:
args = edict({ "logdir" : "./log",
  "env_name": 'sat-v0' ,
  "train_problems_paths": './data/uf50-218-tvt/train' ,
  "eval_problems_paths": './data/uf50-218-tvt/validation' ,
  "lr": 0.00002 ,
  "bsize": 64 ,
  "buffer_size": 20000 ,
  "eps_init":  1.0 ,
  "eps_final": 0.01 ,
  "gamma": 0.99 ,
  "eps_decay_steps": 30000 ,
  'debug':False,
  "batch_updates": 50000 ,
  "history_len": 1 ,
  "init_exploration-steps": 5000 ,
  "step_freq": 4 ,
  "target_update-freq": 10 ,
  "loss": 'mse',
  "opt": 'adam',
  "save_freq": 500 ,
  "grad_clip": 1 ,
  "grad_clip_norm_type": 2 ,
  "eval_freq": 1000 ,
  "eval_time_limit": 3600  ,
  "core_steps": 4 ,
  "expert_exploration_prob": 0.0 ,
  "priority_alpha": 0.5 ,
  "priority_beta": 0.5 ,
  "e2v_aggregator": sum  ,
  "n_hidden": 1 ,
  "hidden_size": 64 ,
  "decoder_v_out_size": 32 ,
  "decoder_e_out_size": 1 ,
  "decoder_g_out_size": 1 ,
  "encoder_v_out_size": 32 ,
  "encoder_e_out_size": 32 ,
  "encoder_g_out_size": 32 ,
  "core_v_out_size": 64 ,
  "core_e_out_size": 64 ,
  "core_g_out_size": 32 ,
  "activation": 'relu' ,
  "penalty_size": 0.1 ,
  "train_time_max_decisions_allowed": 500 ,
  "test_time_max_decisions_allowed": 500 ,
  "no_max_cap_fill_buffer": True ,
  "lr_scheduler_gamma": 1 ,
  "lr_scheduler_frequency": 3000 ,
  "independent_block_layers": 0 ,
  "no_max_cap_fill_buffer" : True,
  "max_cap_fill_buffer" : False,
  "eval_separately_on_each" : True
})

In [ ]:
import sys
sys.maxsize

9223372036854775807

In [9]:
import numpy as np
import torch

import os
from collections import deque
import pickle
import copy
import yaml

from gqsat.utils import build_argparser, evaluate, make_env
from gqsat.models import EncoderCoreDecoder, SatModel
from gqsat.agents import GraphAgent
from gqsat.learners import GraphLearner
from gqsat.buffer import ReplayGraphBuffer

from tensorboardX import SummaryWriter

In [ ]:
env = make_env(args.train_problems_paths, args, test_mode=False)

[Errno 2] No such file or directory: '/content/drive/My Drive/cse 257 Search & Optimization/GraphQSat/data/uf50-218-tvt/train/METADATA'
No metadata available, that is fine for metadata generator.


In [ ]:
obs = env.reset(args.train_time_max_decisions_allowed)

In [ ]:
obs

In [ ]:
env.S.getClauses()

In [ ]:
env.S.getMetadata()

[50, 218, 0, 218, 0, 0]

In [ ]:
env.isSolved

False

In [ ]:
!python3 add_metadata.py

[Errno 2] No such file or directory: '/content/drive/My Drive/cse 257 Search & Optimization/GraphQSat/data/uf50-218-tvt/validation/METADATA'
No metadata available, that is fine for metadata generator.
Testing agent <MiniSAT Agent>... with_restarts is set to False
Traceback (most recent call last):
  File "add_metadata.py", line 61, in <module>
    results, args = main()
  File "add_metadata.py", line 42, in main
    observation = env.reset()
  File "/content/drive/My Drive/cse 257 Search & Optimization/GraphQSat/minisat/minisat/gym/MiniSATEnv.py", line 223, in reset
    self.S = GymSolver(self.curr_problem, self.with_restarts, max_decisions_cap)
  File "/content/drive/My Drive/cse 257 Search & Optimization/GraphQSat/minisat/minisat/gym/GymSolver.py", line 127, in __init__
    this = _GymSolver.new_GymSolver(arg2, arg3, arg4)
OverflowError: in method 'new_GymSolver', argument 3 of type 'int'


In [ ]:
type(None)

NoneType

In [ ]:
a = False

In [ ]:
type(a)

bool

In [ ]:
!./train.sh

In [ ]:
!python3 --version

Python 3.7.10


In [9]:
from gqsat.utils import build_argparser, evaluate, make_env

In [ ]:
!python --version

Python 3.7.10


In [10]:
import os
import gym, minisat  # you need the latter to run __init__.py and register the environment.
from collections import defaultdict
from gqsat.utils import build_argparser
from gqsat.agents import MiniSATAgent

In [11]:
DEBUG_ROLLOUTS = 10  # if --debug flag is present, run this many of rollouts, not the whole problem folder


def main():
    #parser = build_argparser()
    #args = #parser.parse_args()
    # key is the name of the problem file, value is a list with two values [minisat_steps_no_restarts, minisat_steps_with_restarts]
    results = defaultdict(list)

    for with_restarts in [ True, False]:
        env = gym.make(
            "sat-v0",
            args=args,
            problems_paths=args.eval_problems_paths,
            test_mode=True,
            with_restarts=with_restarts,
        )
        agent = MiniSATAgent()
        print(f"Testing agent {agent}... with_restarts is set to {with_restarts}")
        print(type(with_restarts))
        pr = 0
        while env.test_to != 0 or pr == 0:
            observation = env.reset(214744348)
            done = False
            while not done:
                action = agent.act(observation)
                observation, reward, done, info = env.step(action, dummy=True)
            print(
                f'Rollout {pr+1}, steps {env.step_ctr}, num_restarts {info["num_restarts"]}.'
            )
            results[env.curr_problem].append(env.step_ctr)
            pr += 1
            if args.debug and pr >= DEBUG_ROLLOUTS:
                break
        env.close()
    return results, args


In [12]:
from os import path

In [ ]:
!ls

add_metadata.py  docker_run.sh	gqsat			     minisat
data		 dqn.py		graph-env		     README.md
docker_build.sh  evaluate.py	GraphQSAT_driver_code.ipynb  runs
Dockerfile	 evaluate.sh	LICENSE			     train.sh


In [ ]:
if __name__ == "__main__":
    results, args = main()
    for pdir in args.eval_problems_paths.split(":"):
        with open(os.path.join(pdir, "METADATA"), "w") as f:
            for el in sorted(results.keys()):
                cur_dir, pname = path.split(el)
                if path.realpath(pdir) == path.realpath(cur_dir):
                    # no restarts/with restarts
                    f.write(f"{pname},{results[el][0]},{results[el][1]}\n")

In [ ]:
sys.maxsize

9223372036854775807

In [ ]:
int(sys.maxsize)

9223372036854775807

In [ ]:
!pwd

/content/drive/My Drive/cse 257 Search & Optimization/GraphQSat


In [ ]:
os.getcwd()

'/content/drive/My Drive/cse 257 Search & Optimization/GraphQSat'

In [ ]:
dir = "/content/drive/My Drive/cse 257 Search & Optimization/GraphQSat/data/uf50-218-tvt/validation"

In [ ]:
dir = os.path.join()

In [ ]:
with open(os.path.join(dir, "METADATA")) as f:
    for l in f:
        print(l)

uf50-0829.cnf,58,58

uf50-083.cnf,61,61

uf50-0830.cnf,42,42

uf50-0832.cnf,18,18

uf50-0839.cnf,23,23

uf50-0841.cnf,49,49

uf50-0848.cnf,63,63

uf50-0862.cnf,40,40

uf50-0877.cnf,41,41

uf50-0888.cnf,43,43



In [23]:
!pwd

/content/drive/My Drive/cse 257 Search & Optimization/GraphQSat_modified


In [57]:
!ls ../GraphQSat/data/uf100-430-tvt/Test

METADATA					uf100-0939.cnf	uf100-096.cnf
minisat_SAT_100_430_wallclock_time_cap_500.pkl	uf100-093.cnf	uf100-0970.cnf
uf100-0909.cnf					uf100-0940.cnf	uf100-0971.cnf
uf100-0910.cnf					uf100-0941.cnf	uf100-0972.cnf
uf100-0911.cnf					uf100-0942.cnf	uf100-0973.cnf
uf100-0912.cnf					uf100-0943.cnf	uf100-0974.cnf
uf100-0913.cnf					uf100-0944.cnf	uf100-0975.cnf
uf100-0914.cnf					uf100-0945.cnf	uf100-0976.cnf
uf100-0915.cnf					uf100-0946.cnf	uf100-0977.cnf
uf100-0916.cnf					uf100-0947.cnf	uf100-0978.cnf
uf100-0917.cnf					uf100-0948.cnf	uf100-0979.cnf
uf100-0918.cnf					uf100-0949.cnf	uf100-097.cnf
uf100-0919.cnf					uf100-094.cnf	uf100-0980.cnf
uf100-091.cnf					uf100-0950.cnf	uf100-0981.cnf
uf100-0920.cnf					uf100-0951.cnf	uf100-0982.cnf
uf100-0921.cnf					uf100-0952.cnf	uf100-0983.cnf
uf100-0922.cnf					uf100-0953.cnf	uf100-0984.cnf
uf100-0923.cnf					uf100-0954.cnf	uf100-0985.cnf
uf100-0924.cnf					uf100-0955.cnf	uf100-0986.cnf
uf100-0925.cnf					uf100-0956.cnf	uf100-0987.cnf
uf1

In [ ]:
!ls ../GraphQSat/data/uf250-1065/ai/hoos/Shortcuts/UF250.1065.100

In [20]:
meta_args = edict({ "logdir" : "./log",
  "env_name": 'sat-v0' ,
  "train_problems_paths": './data/uf50-218-tvt/train' ,
  "eval_problems_paths": "../GraphQSat/data/uf250-1065/ai/hoos/Shortcuts/UF250.1065.100", #'../GraphQSat/data/uf250-1065/UUF250.1065.100/' ,
  "lr": 0.00002 ,
  "bsize": 64 ,
  "no_cuda": False,
  "buffer_size": 20000 ,
  "eps_init":  1.0 ,
  "eps_final": 0.01 ,
  "gamma": 0.99 ,
  "eps_decay_steps": 30000 ,
  'debug':False,
  "batch_updates": 50000 ,
  "history_len": 1 ,
  "init_exploration-steps": 5000 ,
  "step_freq": 4 ,
  "target_update-freq": 10 ,
  "loss": 'mse',
  "opt": 'adam',
  "save_freq": 500 ,
  "grad_clip": 1 ,
  "grad_clip_norm_type": 2 ,
  "eval_freq": 1000 ,
  "eval_time_limit": 3600  ,
  "core_steps": 4 ,
  "expert_exploration_prob": 0.0 ,
  "priority_alpha": 0.5 ,
  "priority_beta": 0.5 ,
  "e2v_aggregator": "sum"  ,
  "n_hidden": 1 ,
  "hidden_size": 64 ,
  "decoder_v_out_size": 32 ,
  "decoder_e_out_size": 1 ,
  "decoder_g_out_size": 1 ,
  "dump_timings_path" : "../GraphQSat/data/uf250-1065/ai/hoos/Shortcuts" , #'../GraphQSat/data/uuf250-1065/UUF250.1065.100/' 
  "encoder_v_out_size": 32 ,
  "encoder_e_out_size": 32 ,
  "encoder_g_out_size": 32 ,
  "core_v_out_size": 64 ,
  "core_e_out_size": 64 ,
  "core_g_out_size": 32 ,
  "activation": 'relu' ,
  "penalty_size": 0.1 ,
  "train_time_max_decisions_allowed": 500 ,
  "test_time_max_decisions_allowed": 500 ,
  "no_max_cap_fill_buffer": True ,
  "lr_scheduler_gamma": 1 ,
  "lr_scheduler_frequency": 3000 ,
  "independent_block_layers": 0 ,
  "no_max_cap_fill_buffer" : True,
  "max_cap_fill_buffer" : False,
  "eval_separately_on_each" : True
})

In [25]:
DEBUG_ROLLOUTS = 10  # if --debug flag is present, run this many of rollouts, not the whole problem folder


def generate_metadata_util(args):
    #parser = build_argparser()
    #args = #parser.parse_args()
    # key is the name of the problem file, value is a list with two values [minisat_steps_no_restarts, minisat_steps_with_restarts]
    results = defaultdict(list)
    propagations = defaultdict(list)
    wallclock_minisat = defaultdict(list)
    for with_restarts in [ False]:
        env = gym.make(
            "sat-v0",
            args=args,
            problems_paths=args.eval_problems_paths,
            test_mode=True,
            with_restarts=with_restarts,
        )
        agent = MiniSATAgent()
        print(f"Testing agent {agent}... with_restarts is set to {with_restarts}")
        print(type(with_restarts))
        pr = 0
        
        while env.test_to != 0 or pr == 0:
            #no_step = 0
            st_time = time.time()
            observation = env.reset(214744348)
            #no_step += 1
            done = False
            
            while not done:
                action = agent.act(observation)
                observation, reward, done, info = env.step(action, dummy=True)
                #no_step += 1
            en_time = time.time()
            print(
                f'Rollout {pr+1}, steps {env.step_ctr}, mean_propagations {env.S.getPropagations() / (env.step_ctr+1)}, num_restarts {info["num_restarts"]}.'
            )
            results[env.curr_problem].append(env.step_ctr)
            propagations[env.curr_problem].append(env.S.getPropagations() / env.step_ctr)
            wallclock_minisat[env.curr_problem].append(en_time - st_time)
            pr += 1
            if args.debug and pr >= DEBUG_ROLLOUTS:
                break
        env.close()
    return results, args, propagations, wallclock_minisat


In [27]:
import time,pickle
from os import path

In [28]:
#def generate_metadata(args):
results, args, propagations, wallclock_minisat = generate_metadata_util(meta_args)
with open(os.path.join(args.dump_timings_path, "wallclock_minisat.pkl"), 'wb') as f:
    pickle.dump(wallclock_minisat, f)
for pdir in args.eval_problems_paths.split(":"):
    with open(os.path.join(pdir, "METADATA"), "w") as f:
        for el in sorted(results.keys()):
            #print("el: {0}".format(el))
            cur_dir, pname = path.split(el)
            if path.realpath(pdir) == path.realpath(cur_dir):
                # no restarts/with restarts
                #print("results[el][0]: {0}, results[el][1]: {1}".format(results[el][0], results[el][1]))
                f.write(f"{pname},{results[el][0]}\n")

    with open(os.path.join(pdir, "PROP"), "w") as f:
        for el in sorted(propagations.keys()):
            #print("el: {0}".format(el))
            cur_dir, pname = path.split(el)
            if path.realpath(pdir) == path.realpath(cur_dir):
                # no restarts/with restarts
                #print("results[el][0]: {0}, results[el][1]: {1}".format(propagations[el][0], propagations[el][1]))
                f.write(f"{pname},{int(propagations[el][0])}\n")

Testing agent <MiniSAT Agent>... with_restarts is set to False
<class 'bool'>
Rollout 1, steps 58082, mean_propagations 37.02408622144173, num_restarts 0.
Rollout 2, steps 10386, mean_propagations 39.017233079811305, num_restarts 0.
Rollout 3, steps 20199, mean_propagations 38.75306930693069, num_restarts 0.
Rollout 4, steps 14006, mean_propagations 38.47212108231599, num_restarts 0.


KeyboardInterrupt: ignored

In [30]:
with open(os.path.join(args.dump_timings_path, "wallclock_minisat.pkl"), 'wb') as f:
    pickle.dump(wallclock_minisat, f)
for pdir in args.eval_problems_paths.split(":"):
    with open(os.path.join(pdir, "METADATA"), "w") as f:
        for el in sorted(results.keys()):
            #print("el: {0}".format(el))
            cur_dir, pname = path.split(el)
            if path.realpath(pdir) == path.realpath(cur_dir):
                # no restarts/with restarts
                #print("results[el][0]: {0}, results[el][1]: {1}".format(results[el][0], results[el][1]))
                f.write(f"{pname},{results[el][0]}\n")

    with open(os.path.join(pdir, "PROP"), "w") as f:
        for el in sorted(propagations.keys()):
            #print("el: {0}".format(el))
            cur_dir, pname = path.split(el)
            if path.realpath(pdir) == path.realpath(cur_dir):
                # no restarts/with restarts
                #print("results[el][0]: {0}, results[el][1]: {1}".format(propagations[el][0], propagations[el][1]))
                f.write(f"{pname},{int(propagations[el][0])}\n")

In [29]:
len(results)

100

In [24]:
import pickle

In [25]:
with open("uf_250.pkl", 'wb') as f:
    pickle.dump(results, f)

In [30]:
from os import path
from os.path import join

In [97]:
generate_metadata(meta_args)

[Errno 2] No such file or directory: '/content/drive/My Drive/cse 257 Search & Optimization/GraphQSat/data/uuf100-430/Test/METADATA'
No metadata available, that is fine for metadata generator.
Testing agent <MiniSAT Agent>... with_restarts is set to False
<class 'bool'>
Rollout 1, steps 638, mean_propagations 19.72300469483568, num_restarts 0.
Rollout 2, steps 973, mean_propagations 19.23100616016427, num_restarts 0.
Rollout 3, steps 514, mean_propagations 20.13009708737864, num_restarts 0.
Rollout 4, steps 409, mean_propagations 18.990243902439026, num_restarts 0.
Rollout 5, steps 535, mean_propagations 19.89179104477612, num_restarts 0.
Rollout 6, steps 786, mean_propagations 19.911054637865313, num_restarts 0.
Rollout 7, steps 549, mean_propagations 19.96727272727273, num_restarts 0.
Rollout 8, steps 537, mean_propagations 19.092936802973977, num_restarts 0.
Rollout 9, steps 583, mean_propagations 19.86472602739726, num_restarts 0.
Rollout 10, steps 602, mean_propagations 19.8457711

In [98]:
dir = "../GraphQSat/data/uuf50-218/Test"
prop = {}

In [99]:
with open(join(dir, "PROP")) as f:
    for l in f:
        k, rscore, msscore = l.split(",")
        prop[k] =[int(rscore), int(msscore)]
        print("problem: {0}   without_restart: {1},   with_restart: {2}".format(k, int(rscore), int(msscore)))

problem: uuf50-01000.cnf   without_restart: 12,   with_restart: 12
problem: uuf50-0900.cnf   without_restart: 9,   with_restart: 9
problem: uuf50-0901.cnf   without_restart: 11,   with_restart: 11
problem: uuf50-0902.cnf   without_restart: 13,   with_restart: 13
problem: uuf50-0903.cnf   without_restart: 13,   with_restart: 13
problem: uuf50-0904.cnf   without_restart: 12,   with_restart: 12
problem: uuf50-0905.cnf   without_restart: 13,   with_restart: 13
problem: uuf50-0906.cnf   without_restart: 12,   with_restart: 12
problem: uuf50-0907.cnf   without_restart: 10,   with_restart: 10
problem: uuf50-0908.cnf   without_restart: 14,   with_restart: 14
problem: uuf50-0909.cnf   without_restart: 12,   with_restart: 12
problem: uuf50-0911.cnf   without_restart: 11,   with_restart: 11
problem: uuf50-0912.cnf   without_restart: 12,   with_restart: 12
problem: uuf50-0913.cnf   without_restart: 13,   with_restart: 13
problem: uuf50-0914.cnf   without_restart: 13,   with_restart: 13
problem: uu

In [16]:
import time

In [19]:
eval_args = edict({ 
  "logdir" : "./log",
  "env_name": 'sat-v0' ,
  "eval_problems_paths": '../GraphQSat/data/uuf250-1065/UUF250.1065.100',
  "eps_final": 0.01 ,
  'model_dir':"../GraphQSat/runs/May14_21-04-22_85516a6ced97",
  'model_checkpoint': 'model_26003.chkp',
  'debug':False,
  'no_cuda':False,
  "eval_time_limit": 100000000000000,
  "core_steps": -1,
  "dump_timings_path": '../GraphQSat/data/uuf250-1065/',
  "test_time_max_decisions_allowed": 500 ,
  "eval_separately_on_each":True, 
  "compare_with_restarts" : False
})

In [20]:
def eval_exp(args):
    
    eval_args = args
    
    with open(os.path.join(eval_args.model_dir, "status.yaml"), "r") as f:
        train_status = yaml.load(f, Loader=yaml.Loader)
        args = train_status["args"]

    # use same args used for training and overwrite them with those asked for eval
    for k, v in vars(eval_args).items():
        setattr(args, k, v)

    args.device = (
        torch.device("cpu")
        if args.no_cuda or not torch.cuda.is_available()
        else torch.device("cuda")
    )
    net = SatModel.load_from_yaml(os.path.join(args.model_dir, "model.yaml")).to(
        args.device
    )

    # modify core steps for the eval as requested
    if args.core_steps != -1:
        # -1 if use the same as for training
        net.steps = args.core_steps

    net.load_state_dict(
        torch.load(os.path.join(args.model_dir, args.model_checkpoint)), strict=False
    )

    agent = GraphAgent(net, args)

    st_time = time.time()
    scores, eval_metadata, _ = evaluate(agent, args)
    end_time = time.time()

    print(
        f"Evaluation is over. It took {end_time - st_time} seconds for the whole procedure"
    )

    # with open("../eval_results.pkl", "wb") as f:
    #     pickle.dump(scores, f)res_list

    for pset, pset_res in scores.items():
        res_list = [el for el in pset_res.values()]
        print(f"Results for {pset}")
        print(
            f"median_relative_score: {np.nanmedian(res_list)}, mean_relative_score: {np.mean(res_list)}, max_score: {np.max(res_list)}, min_score: {np.min(res_list)}"
        )

In [87]:
!pwd

/content/drive/My Drive/cse 257 Search & Optimization/GraphQSat_modified


In [21]:
eval_exp(eval_args)

RuntimeError: ignored

In [39]:
!pwd

/content/drive/My Drive/cse 257 Search & Optimization/GraphQSat_modified


In [47]:
!ls ../GraphQSat/data/uf-218-tvt/eval-problems-paths

METADATA       uf50-0926.cnf  uf50-0946.cnf  uf50-0965.cnf  uf50-0984.cnf
PROP	       uf50-0927.cnf  uf50-0947.cnf  uf50-0966.cnf  uf50-0985.cnf
uf50-0665.cnf  uf50-0928.cnf  uf50-0948.cnf  uf50-0967.cnf  uf50-0986.cnf
uf50-0681.cnf  uf50-0929.cnf  uf50-0949.cnf  uf50-0968.cnf  uf50-0987.cnf
uf50-0698.cnf  uf50-0930.cnf  uf50-094.cnf   uf50-0969.cnf  uf50-0988.cnf
uf50-0713.cnf  uf50-0931.cnf  uf50-0950.cnf  uf50-096.cnf   uf50-0989.cnf
uf50-073.cnf   uf50-0932.cnf  uf50-0951.cnf  uf50-0970.cnf  uf50-098.cnf
uf50-0746.cnf  uf50-0933.cnf  uf50-0952.cnf  uf50-0971.cnf  uf50-0990.cnf
uf50-0762.cnf  uf50-0934.cnf  uf50-0953.cnf  uf50-0972.cnf  uf50-0991.cnf
uf50-0779.cnf  uf50-0935.cnf  uf50-0954.cnf  uf50-0973.cnf  uf50-0992.cnf
uf50-0795.cnf  uf50-0936.cnf  uf50-0955.cnf  uf50-0974.cnf  uf50-0993.cnf
uf50-0810.cnf  uf50-0937.cnf  uf50-0956.cnf  uf50-0975.cnf  uf50-0994.cnf
uf50-0827.cnf  uf50-0938.cnf  uf50-0957.cnf  uf50-0976.cnf  uf50-0995.cnf
uf50-0843.cnf  uf50-0939.cnf  uf50-0958.cn

In [49]:
import pickle

In [105]:
with open("../GraphQSat/data/uuf100-430/Test/.._GraphQSat_data_uuf100-430_Test_cap_500_propagations.pkl", 'rb') as f:
    prop = pickle.load(f)

In [106]:
for key, val in prop.items():
    print("{0} -> {1}".format(key.split("/")[-1], val))

uuf100-0902.cnf -> 20
uuf100-0901.cnf -> 20
uuf100-0904.cnf -> 19
uuf100-0905.cnf -> 18
uuf100-0903.cnf -> 20
uuf100-0906.cnf -> 19
uuf100-0907.cnf -> 19
uuf100-0909.cnf -> 20
uuf100-0908.cnf -> 21
uuf100-0910.cnf -> 20
uuf100-0911.cnf -> 18
uuf100-0914.cnf -> 20
uuf100-0912.cnf -> 19
uuf100-0913.cnf -> 18
uuf100-0915.cnf -> 19
uuf100-0916.cnf -> 20
uuf100-0917.cnf -> 19
uuf100-0918.cnf -> 20
uuf100-0919.cnf -> 19
uuf100-0920.cnf -> 20
uuf100-0921.cnf -> 20
uuf100-0922.cnf -> 18
uuf100-0923.cnf -> 19
uuf100-0925.cnf -> 19
uuf100-0924.cnf -> 20
uuf100-0926.cnf -> 20
uuf100-0927.cnf -> 20
uuf100-0928.cnf -> 18
uuf100-0929.cnf -> 20
uuf100-0930.cnf -> 20
uuf100-0933.cnf -> 18
uuf100-0931.cnf -> 20
uuf100-0932.cnf -> 20
uuf100-0935.cnf -> 20
uuf100-0936.cnf -> 20
uuf100-0934.cnf -> 18
uuf100-0939.cnf -> 19
uuf100-0938.cnf -> 20
uuf100-0937.cnf -> 20
uuf100-0942.cnf -> 18
uuf100-0941.cnf -> 19
uuf100-0940.cnf -> 19
uuf100-0943.cnf -> 19
uuf100-0944.cnf -> 20
uuf100-0946.cnf -> 21
uuf100-094